### Исходное задание 
1. нужно скачивать информацию о блоке и сохранять (чтобы каждый раз не тянуть данные, а если нужно то подгрузить, лучше в несколько потоков)
http://93.125.26.210:34657/block?height=300
2. далее распарсить итеративно все блоки и сохранить информацию о коммитах для каждого валидатора для каждого блока
3. нарисовать графики для валидаторов, график лайфтайма
4. расчитать вознаграждение для каждого исходя из линейно возрастающей награды за каждый блок
5. выдать таблицу - валидатор, баланс

In [8]:
import pandas as pd

In [116]:
import json

In [9]:
from tqdm import tqdm_notebook

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

In [12]:
import plotly.graph_objects as go

In [13]:
import numpy as np

In [6]:
# validators_df = pd.read_csv("euler_validators_2.csv", usecols=["block", "validators"])

In [7]:
# validators_df[["block", "validators"]].to_csv("euler_validators.csv")

In [86]:
validators_df = pd.read_csv("val_test.csv") #pd.read_csv("euler_validators2.csv")

In [87]:
validators_df.columns = ['index', 'block', 'validators']

In [88]:
validators_df["validators"] = validators_df["validators"].apply(
    lambda x: eval(x))

In [89]:
validators_df

,index,block,validators
0,0,2000000,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
1,1,2000001,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
2,2,2000002,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
3,3,2000003,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
4,4,2000004,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
5,5,2000005,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
6,6,2000006,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
7,7,2000007,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
8,8,2000008,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."
9,9,2000009,"{er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,..."


In [10]:
# validators_df["validators"] = validators_df["validators"].apply(lambda x: set(x))

In [90]:
all_validators = set([v for v_list in validators_df["validators"].tolist() for v in v_list])

In [91]:
all_validators

{'1hAgw0k5iFwVNRDmoRG6ZkwGqxpKDJsBVNYKfkceRKg=',
 '6cD8SSQHOQBJOZcem3ycB+sW//DIAm+gScr7huJzOsk=',
 'e4pHUC9NCi2oNTitTaZfFFRrvjg0AVXx8u1ts+0Pz6U=',
 'er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=',
 'hPZIQAs0nWDV6gSEurwZQ/6mb2tNYAPXKQbvu0GXx2Y=',
 'pmJaelb4hFa0yPtQmpeYVmVu30/UDKFztkeTXnhst9g=',
 'wTJYthVYdrIDOKZrFOoLLMmM6YJUYGVWZBK29m7jtS4=',
 'yOl+PMVvwYSlhdjimjt7juK8jyZueh87NamQeDBvN1M='}

### Визуализация работы валидаторов

In [92]:
# Последний актуальный блок
MAX_BLOCK = validators_df["block"].max()
MAX_BLOCK

2009999

In [93]:
validators_df["rounded_block"] = validators_df["block"].round(-4)

In [94]:
validators_df["rounded_block"].nunique()

2

In [95]:
x = validators_df["rounded_block"].unique().tolist()

In [96]:
all_y = []
labels = []
validator_traces = {}
for validator in tqdm_notebook(all_validators):
    validators_df["validator_mask"] = validators_df["validators"].apply(lambda x: validator in x).astype(float)
    y = validators_df.groupby("rounded_block")["validator_mask"].sum().tolist()
    all_y.append(y)
    labels.append(validator)
    validator_traces[validator] = y

In [97]:
import matplotlib
cmap = matplotlib.cm.get_cmap('jet')

In [98]:
validator_colors = {
    validator: "rgba({})".format(", ".join([str(int(color * 255)) for color in cmap(np.sum(trace) / MAX_BLOCK)]))
    for validator, trace in validator_traces.items()
}

In [99]:
fig = go.Figure()

for validator, y in validator_traces.items():
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        name=validator[0:5] + "...",
        line=dict(width=0.5, color=validator_colors[validator]),
        stackgroup='one',
        groupnorm='percent'
    ))

fig.update_layout(
    showlegend=True,
    xaxis_type='linear',
    yaxis=dict(
        type='linear',
        range=[1, 100],
        ticksuffix='%'))

fig.show()

### Расчет вознаграждения для каждого валидатора

In [100]:
# Количество токенов, участвующее в раздаче
AMOUNT_OF_TOKENS = 10000000

In [101]:
# Коэффициенты прямой для вычисления вознаграждения
K = 1
B = 0

Вопрос: как влияют эти коэффициенты на вознаграждение валидаторов?

In [102]:
def linear(block):
    return K * block + B

In [103]:
validators_df["total_reward_part"] = validators_df["block"].apply(lambda x: linear(x)) 

In [104]:
validators_df["total_validators"] = validators_df["validators"].apply(lambda x: len(x))

In [105]:
validators_df["validator_reward_part"] = validators_df["total_reward_part"] / validators_df["total_validators"] / validators_df["total_reward_part"].sum()

In [106]:
rewards = {}

for validator in tqdm_notebook(all_validators):
    validator_reward = validators_df[validators_df["validators"].apply(lambda x: validator in x)]["validator_reward_part"].sum()
    rewards[validator] = validator_reward * AMOUNT_OF_TOKENS

In [107]:
balances_df = pd.Series(rewards).to_frame().reset_index().rename(columns={0: "balance", "index": "address"}).set_index("address").sort_values("balance", ascending=False)

In [126]:
balances_df = balances_df.reset_index()

In [127]:
balances_df

,address,balance
0,er5nOfcT583OQP/QeLR/Wqpl+ttrr3seCwmiZpkTv6U=,1250000.0
1,e4pHUC9NCi2oNTitTaZfFFRrvjg0AVXx8u1ts+0Pz6U=,1250000.0
2,1hAgw0k5iFwVNRDmoRG6ZkwGqxpKDJsBVNYKfkceRKg=,1250000.0
3,6cD8SSQHOQBJOZcem3ycB+sW//DIAm+gScr7huJzOsk=,1250000.0
4,hPZIQAs0nWDV6gSEurwZQ/6mb2tNYAPXKQbvu0GXx2Y=,1250000.0
5,yOl+PMVvwYSlhdjimjt7juK8jyZueh87NamQeDBvN1M=,1250000.0
6,wTJYthVYdrIDOKZrFOoLLMmM6YJUYGVWZBK29m7jtS4=,1250000.0
7,pmJaelb4hFa0yPtQmpeYVmVu30/UDKFztkeTXnhst9g=,1250000.0


In [ ]:
#balances_df.sort_values("balance").plot(kind="barh", figsize=(10, 25))

In [130]:
with open('balance.json', 'w') as json_file:
    json.dump(dict(zip([i for i in balances_df['address']],
                       [i for i in balances_df['balance']])), 
              json_file)
